In [11]:
# MYIP Application Source Code

import cv2
import numpy as np
import random

###Overlay 함수#######################################################################


def overlay(img, glass, pos):
    sx = pos[0]
    ex = pos[0] + glass.shape[1]
    sy = pos[1]
    ey = pos[1] + glass.shape[0]

    if sx < 0 or ex > img.shape[1] or sy < 0 or ey > img.shape[0]:
        return
    img1 = img[sy:ey, sx:ex]
    img2 = glass[:, :, 0:3]
    alpha = 1.0 - (glass[:, :, 3] / 255.0)

    img1[..., 0] = (img1[..., 0] * alpha + img2[..., 0] * (1 - alpha)).astype(
        np.uint8)
    img1[..., 1] = (img1[..., 1] * alpha + img2[..., 1] * (1 - alpha)).astype(
        np.uint8)
    img1[..., 2] = (img1[..., 2] * alpha + img2[..., 2] * (1 - alpha)).astype(
        np.uint8)


######################################################################################

cap = cv2.VideoCapture('./img/presentation.mp4')
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

face_classifier = cv2.CascadeClassifier(
    './xml/haarcascade_frontalface_alt.xml')
eyes_classifier = cv2.CascadeClassifier('./xml/haarcascade_eye.xml')

glass = cv2.imread('./img/sunglasses_PNG155.png', cv2.IMREAD_UNCHANGED)
glass2 = cv2.imread('./img/alienglass.png', cv2.IMREAD_UNCHANGED)
bgimg = cv2.imread('./img/bgimg.jpg', cv2.IMREAD_COLOR)
bgimg = cv2.resize(bgimg, dsize=(640, 360), interpolation=cv2.INTER_AREA)

flocs = ['heart', 'yukku', 'check', 'music', 'textballoon']

timglist = [None]  # size will be 6.

for item in flocs:
    tempimg = cv2.imread('./img/' + item + '.png', cv2.IMREAD_UNCHANGED)
    timglist.append(tempimg)
tidx = 0

# this is Transparent Images List. (for Glasses!!!)
list = [
    None, (glass, glass.shape[0], glass.shape[1], 820, 740, 2450, 740),
    (glass2, glass2.shape[0], glass2.shape[1], 385, 1285, 1361, 1285)
]
idx = 0

lastfacepos = ()
lasteyespos = ()


def timgBarChange(event):
    global tidx
    tidx = event


def glassBarChange(event):
    global idx
    idx = event


xi, yi = 0, 0
xl, yl = 0, 0


def mouseEventHandler(
    event, x, y, flags, param
):  # you should draw in 'bgimg' !!!! with NO RESET of bgimg. (maybe added in the future)
    global xi, yi, xl, yl  # x init, x last.
    if tidx == 0:
        return
    if event == cv2.EVENT_LBUTTONDOWN:
        xi, yi = x, y
    elif event == cv2.EVENT_LBUTTONUP:  # you should draw in here..
        xl, yl = x, y
        ds = (abs(xl - xi), abs(yl - yi))
        result = cv2.resize(timglist[tidx], ds, interpolation=cv2.INTER_AREA)

        # 여기서 overlay 함수를 호출해야합니다.
        overlay(bgimg, result, (xi, yi))


cv2.namedWindow('Make Yourself In Party')
cv2.createTrackbar('T.Image', 'Make Yourself In Party', 0, 5, timgBarChange)
cv2.createTrackbar('Glasses', 'Make Yourself In Party', 0, 2, glassBarChange)
cv2.setMouseCallback('Make Yourself In Party', mouseEventHandler)

curCircleSize = 0
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

rW = random.randrange(1, frameWidth)
rH = random.randrange(1, frameHeight)

while True:
    ret, frame = cap.read()

    #프레임마다 색이 바뀜
    rB = random.randrange(0, 256)
    rG = random.randrange(0, 256)
    rR = random.randrange(0, 256)

    frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    bgimg3 = np.copy(bgimg)  # 똑같은거 0.5+0.5 하면 1이므로 AddWeighted의 결과는 똑같을 것이다.

    cv2.circle(bgimg3, (rW, rH), curCircleSize, (rB, rG, rR), -1)
    curCircleSize += 7
    if curCircleSize > int(frameWidth / 5):
        curCircleSize = 0
        rW = random.randrange(1, frameWidth)
        rH = random.randrange(1, frameHeight)

    bgimg = cv2.addWeighted(bgimg, 0.5, bgimg3, 0.5, 0)

    if idx != 0:
        timg = list[idx]
        ew, eh = timg[1], timg[2]
        ex1, ey1 = timg[3], timg[4]
        ex2, ey2 = timg[5], timg[6]

    if not ret:
        print('Video Capture Failure')
        break

    faces = face_classifier.detectMultiScale(frame,
                                             scaleFactor=1.05,
                                             minSize=(100, 100),
                                             maxSize=(400, 400))

    if len(faces
           ) == 0 and len(lastfacepos) != 0:  # this will make GrabCut steady.
        faces = lastfacepos

    for (x, y, w, h) in faces:
        lastfacepos = faces
        ROI = (int(x - w * 3 / 4), int(y - h * 3 / 4), int(2.5 * w),
               int(2.5 * h))

        #for debugging
        #cv2.rectangle(frame,(x,y,w,h),(255,0,255),1)
        #cv2.rectangle(frame,ROI,(0,255,0),2)

        faceR01 = frame[y:y + h // 2, x:x + w]
        eyes = eyes_classifier.detectMultiScale(faceR01)

        mask = np.zeros(frame.shape[:2], np.uint8)
        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)

        cv2.grabCut(frame, mask, ROI, bgdModel, fgdModel, 3,
                    cv2.GC_INIT_WITH_RECT)  # Performance Issue (iterCount)
        mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')
        mask3 = np.where((mask == 0) | (mask == 2), 1, 0).astype('uint8')
        bgimg2 = bgimg * mask3[:, :, np.newaxis]
        frame = frame * mask2[:, :, np.newaxis]
        frame = cv2.bitwise_or(frame, bgimg2)

        #for debugging
        #for(ex,ey,ew,eh) in eyes:
        #cv2.rectangle(faceR01,(ex,ey),(ex+ew,ey+eh),(0,255,0),1)

        if len(eyes) != 2 and len(
                lasteyespos) != 0:  # this will make overlay steady.
            eyes = lasteyespos
        else:
            lasteyespos = eyes

        if len(eyes) == 0:
            continue

        # 눈 중앙 위치 계산
        x1 = x + eyes[0][0] + eyes[0][2] // 2
        y1 = y + eyes[0][1] + eyes[0][3] // 2

        x2 = x + eyes[1][0] + eyes[1][2] // 2
        y2 = y + eyes[1][1] + eyes[1][3] // 2

        if x1 > x2:
            x1, y1, x2, y2 = x2, y2, x1, y1

        # 비율 계산 후 투명이미지 리사이즈 후 Overlay
        if idx != 0:
            fx = (x2 - x1) / (ex2 - ex1)
            glass2 = cv2.resize(timg[0], (0, 0),
                                fx=fx,
                                fy=fx,
                                interpolation=cv2.INTER_AREA)
            pos = (x1 - int(ex1 * fx), (y1 - int(ey1 * fx)))
            overlay(frame, glass2, pos)

    cv2.imshow('Make Yourself In Party', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()